In [ ]:
!pip install numpy

In [ ]:
import numpy as np
from numba import cuda
import time

# Função para computar o scan paralelo Blelloch
@cuda.jit
def blelloch_scan_up_sweep(arr, step):
    tid = cuda.grid(1)
    if tid % (2 * step) == 0:
        arr[tid + 2 * step - 1] += arr[tid + step - 1]

@cuda.jit
def blelloch_scan_down_sweep(arr, step):
    tid = cuda.grid(1)
    if tid % (2 * step) == 0:
        t = arr[tid + step - 1]
        arr[tid + step - 1] = arr[tid + 2 * step - 1]
        arr[tid + 2 * step - 1] += t

# Função principal para execução do Blelloch scan
def blelloch_scan(arr):
    n = len(arr)
    threads_per_block = 128
    blocks = (n + threads_per_block - 1) // threads_per_block

    # Up-sweep (redução)
    step = 1
    while step < n:
        blelloch_scan_up_sweep[blocks, threads_per_block](arr, step)
        step *= 2

    # Definir o último elemento para zero
    arr[n - 1] = 0

    # Down-sweep (distribuição)
    step = n // 2
    while step > 0:
        blelloch_scan_down_sweep[blocks, threads_per_block](arr, step)
        step //= 2

# Testar para tamanhos diferentes de arrays e medir tempo de execução e trabalho
array_sizes = [100, 1000, 10000, 100000, 1000000, 10000000]
for size in array_sizes:
    # Inicializar array com valores aleatórios
    arr = np.random.randint(1, 10, size).astype(np.int32)
    d_arr = cuda.to_device(arr)

    # Tempo inicial
    start = time.time()

    # Executar o scan
    blelloch_scan(d_arr)

    # Sincronizar e medir tempo
    cuda.synchronize()
    end = time.time()

    # Calcular trabalho realizado e número de etapas
    time_taken = end - start
    work_done = np.sum(arr)  # como exemplo simplificado, usando a soma inicial do array
    steps_needed = int(np.log2(size))

    print(f"Tamanho do array: {size}")
    print(f"Tempo de execução: {time_taken:.6f} segundos")
    print(f"Trabalho realizado: {work_done}")
    print(f"Etapas necessárias: {steps_needed}")
    print("-" * 40)
